# Business KPI forecasting s reálnými daty

In [ ]:
import pandas as pd

# Dataset: business_kpi_forecasting.tsv
df_business = pd.read_csv('/content/drive/MyDrive/service-brain-digital-horizon/WORKSPACE/2025-07-25/business_kpi_forecasting.tsv', sep='\t')
df_business['date'] = pd.to_datetime(df_business['date'])
df_business = df_business.set_index('date').sort_index()

display(df_business.head())

# Gemini prompt pro business analýzu
prompt = """
Analyzuj tento business dataset:
1. Identifikuj klíčové trendy v daily_revenue
2. Vysvětli vztah mezi marketing_spend a new_customers
3. Navrhni prediktivní model pro cash_flow_impact
4. Definuj business kritéria pro alerting systém
5. Doporuč KPI pro automatické rozhodování
"""

print(prompt)

In [ ]:
# Dataset: business_kpi_forecasting.tsv
def multi_horizon_forecast(data, target_col, horizons=[7, 14, 30]):
    """Prognóza pro různé časové horizonty"""
    forecasts = {}

    for horizon in horizons:
        # Prophet model pro business data
        from prophet import Prophet

        prophet_data = data.reset_index()
        prophet_data['ds'] = pd.to_datetime(prophet_data['date'])
        prophet_data['y'] = prophet_data[target_col]
        prophet_data = prophet_data[['ds', 'y']]

        # Prophet model pro business data
        # Prophet generally requires more data (ideally a year or more) to accurately model seasonality and trends.
        # With only 46 data points, the forecast might not be very accurate.
        model = Prophet(
            yearly_seasonality=False,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.01,
            seasonality_prior_scale=0.1
        )
        model.add_seasonality(name='weekly', period=7, fourier_order=3)
        model.fit(prophet_data)

        future = model.make_future_dataframe(periods=horizon)
        forecast = model.predict(future)

        forecasts[f'{horizon}d'] = forecast.tail(horizon)

    return forecasts

# Revenue forecasting
revenue_forecasts = multi_horizon_forecast(df_business, 'daily_revenue')

In [24]:
# display(revenue_forecasts)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_forecast(data, forecast, target_col, horizon):
    """Vizualizace prognózy"""
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=data.index, y=data[target_col], label='Historická data')
    sns.lineplot(x=forecast['ds'], y=forecast['yhat'], label=f'Prognóza ({horizon})', color='orange')
    plt.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='orange', alpha=0.2)
    plt.title(f'Prognóza {target_col} pro {horizon}')
    plt.xlabel('Datum')
    plt.ylabel(target_col)
    plt.legend()
    plt.show()

# Plot forecasts for each horizon
for horizon, forecast_data in revenue_forecasts.items():
    plot_forecast(df_business, forecast_data, 'daily_revenue', horizon)

# Automatizace business rozhodování

## Inventory Management Automation

In [ ]:
import numpy as np
from scipy.stats import norm

# Dataset: business_kpi_forecasting.tsv
# EOQ (Economic Order Quantity) calculation
def calculate_eoq(annual_demand, order_cost, holding_cost):
    return np.sqrt((2 * annual_demand * order_cost) / holding_cost)

# Safety stock calculation
def calculate_safety_stock(demand_std, lead_time, service_level=0.95):
    z_score = norm.ppf(service_level)
    return z_score * demand_std * np.sqrt(lead_time)

# Reorder point
avg_daily_demand = df_business['inventory_turnover'].rolling(30).mean().iloc[-1]
demand_std = df_business['inventory_turnover'].rolling(30).std().iloc[-1]
lead_time = 3  # dny

safety_stock = calculate_safety_stock(demand_std, lead_time)
reorder_point = (avg_daily_demand * lead_time) + safety_stock

# Decision logic
current_inventory = 100  # example current level

def automated_inventory_decision(current_inventory, reorder_point, safety_stock, avg_daily_demand):
    """Automatizované rozhodování o zásobách"""

    if current_inventory <= reorder_point:
        annual_demand = avg_daily_demand * 365
        eoq = calculate_eoq(annual_demand, order_cost=500, holding_cost=50)

        decision = {
            'action': 'REORDER',
            'quantity': eoq,
            'reason': f'Inventory ({current_inventory}) below reorder point ({reorder_point:.1f})',
            'urgency': 'HIGH' if current_inventory < safety_stock else 'NORMAL'
        }
    else:
        decision = {
            'action': 'NO_ACTION',
            'quantity': 0,
            'reason': f'Inventory ({current_inventory}) above reorder point ({reorder_point:.1f})',
            'urgency': 'LOW'
        }

    return decision


# Implementace
inventory_decision = automated_inventory_decision(current_inventory, reorder_point, safety_stock, avg_daily_demand)
print(f"📦 INVENTORY DECISION: {inventory_decision}")

In [ ]:
import matplotlib.pyplot as plt

# Prepare data for visualization
labels = ['Safety Stock', 'Reorder Point', 'Current Inventory']
levels = [safety_stock, reorder_point, current_inventory]

# Create the bar chart
plt.figure(figsize=(8, 5))
plt.bar(labels, levels, color=['red', 'orange', 'green'])
plt.ylabel('Inventory Level')
plt.title('Inventory Levels vs. Reorder Points')
plt.show()

## Marketing Budget Optimization

In [ ]:
# Dataset: business_kpi_forecasting.tsv
def optimize_marketing_budget(data, total_budget, channels=['digital', 'traditional', 'social']):
    """Optimalizace marketingového rozpočtu"""

    # Historical ROI by channel (simulace)
    channel_roi = {
        'digital': data['roi_marketing'].mean() * 1.2,
        'traditional': data['roi_marketing'].mean() * 0.8,
        'social': data['roi_marketing'].mean() * 1.1
    }

    # Budget allocation based on ROI
    total_roi_weight = sum(channel_roi.values())
    allocation = {}

    for channel, roi in channel_roi.items():
        weight = roi / total_roi_weight
        allocation[channel] = {
            'budget': total_budget * weight,
            'roi': roi,
            'expected_return': total_budget * weight * roi
        }

    return allocation

# Budget optimization
budget_allocation = optimize_marketing_budget(df_business, total_budget=50000)
print("💰 OPTIMALIZED BUDGET ALLOCATION:")
for channel, details in budget_allocation.items():
    print(f"{channel.upper()}: ${details['budget']:.0f} (ROI: {details['roi']:.2f})")

In [ ]:
# Prepare data for visualization
channels = list(budget_allocation.keys())
budgets = [allocation['budget'] for allocation in budget_allocation.values()]

# Create the bar chart
plt.figure(figsize=(8, 5))
plt.bar(channels, budgets, color=['blue', 'red', 'green'])
plt.ylabel('Allocated Budget ($)')
plt.title('Optimized Marketing Budget Allocation')
plt.show()